# physik690 - Bachelorarbeit

## Initial setup

### Requirements

In [ ]:
# Standard imports for computational physics
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter
from scipy.stats import linregress
import pandas as pd
from enum import Enum
import sqlite3
import scienceplots

# Import schemas generated by the Flatbuffers compiler
from schemas.python.schemas.Vector import Vector

# Pretty styling for graphs
plt.style.use(['science', 'grid', 'scatter'])
plt.rcParams["figure.figsize"] = (10, 6)

dpi = 300

### Observables

In [ ]:
class Algorithm(Enum):
    Metropolis = 0
    Wolff = 1

class Observable(Enum):
    Energy = 0
    EnergySquare = 1
    Magnetization = 2
    MagnetizationSquare = 3
    SpecificHeat = 4
    MagneticSusceptibility = 5
    HelicityModulusIntermediate = 6
    HelicityModulus = 7
    ClusterSize = 8

    def name(self) -> str:
        match self:
            case Observable.Energy:
                return "Energy"
            case Observable.EnergySquare:
                return "Energy squared"
            case Observable.Magnetization:
                return "Magnetization"
            case Observable.MagnetizationSquare:
                return "Magnetization squared"
            case Observable.SpecificHeat:
                return "Specific heat"
            case Observable.MagneticSusceptibility:
                return "Magnetic susceptibility"
            case Observable.HelicityModulusIntermediate:
                return "Helicity modulus intermediate"
            case Observable.HelicityModulus:
                return "Helicity modulus"
            case Observable.ClusterSize:
                return "Cluster size"

    def symbol(self) -> str:
        match self:
            case Observable.Energy:
                return "E"
            case Observable.EnergySquare:
                return "E^2"
            case Observable.Magnetization:
                return "M"
            case Observable.MagnetizationSquare:
                return "M^2"
            case Observable.SpecificHeat:
                return "C_V"
            case Observable.MagneticSusceptibility:
                return "\\chi"
            case Observable.HelicityModulusIntermediate:
                return "\\Gamma"
            case Observable.HelicityModulus:
                return "\\Gamma"
            case Observable.ClusterSize:
                return "S_{\\text{Cluster}}"

### Helper for plotting observables

In [ ]:
def plot_observable_per_spin(con: sqlite3.Connection, simulation_id: int, alg: Algorithm, observable: Observable, legend_loc: str = 'best') -> tuple[plt.Figure, plt.Axes]:
    fig, ax = plt.subplots()
    for size in pd.read_sql_query(f'SELECT DISTINCT lattice_size FROM configurations c INNER JOIN metadata m ON c.metadata_id = m.metadata_id AND m.algorithm = {alg.value} WHERE c.simulation_id = {simulation_id}', con)['lattice_size']:
        data = pd.read_sql_query(f'SELECT temperature, e.mean, e.std_dev FROM configurations c INNER JOIN metadata m ON c.metadata_id = m.metadata_id AND m.algorithm = {alg.value} INNER JOIN estimates e ON e.configuration_id = c.configuration_id AND e.type_id = {observable.value} WHERE c.simulation_id = \'{simulation_id}\' AND c.lattice_size = \'{size}\' ORDER BY temperature', con)
        ax.errorbar(data['temperature'], data['mean'], label=f'$N = {size} \\times {size}$')

        ax.set_title(f'{observable.name()} per spin for a 2D lattice using the {alg.name} algorithm')
        ax.set_xlabel('Temperature $T / \\frac{J}{k_B}$')
        ax.set_ylabel(f'{observable.name()} per spin ${observable.symbol()}$')

    ax.legend(loc=legend_loc)
    return fig, ax

def get_observable_max(con: sqlite3.Connection, simulation_id: int, alg: Algorithm, observable: Observable) -> pd.DataFrame:
    return pd.read_sql_query(f'SELECT lattice_size, temperature, MAX(mean) FROM configurations c INNER JOIN metadata m ON c.metadata_id = m.metadata_id AND m.algorithm = {alg.value} INNER JOIN estimates e ON e.configuration_id = c.configuration_id AND e.type_id = {observable.value} WHERE c.simulation_id = \'{simulation_id}\' AND c.temperature BETWEEN 0.8 AND 1.2 AND lattice_size >= 20 GROUP BY c.lattice_size', con)

### Helper for plotting integrated autocorrelation

In [ ]:
def plot_observable_integrated_autocorrelation(con: sqlite3.Connection, simulation_id: int, alg: Algorithm, observable: Observable) -> tuple[plt.Figure, plt.Axes]:
    fig, ax = plt.subplots()
    for size in pd.read_sql_query(f'SELECT DISTINCT lattice_size FROM configurations c INNER JOIN metadata m ON c.metadata_id = m.metadata_id AND m.algorithm = {alg.value} WHERE c.simulation_id = {simulation_id}', con)['lattice_size']:
        data = pd.read_sql_query(f'SELECT temperature, tau FROM configurations c INNER JOIN metadata m ON c.metadata_id = m.metadata_id AND m.algorithm = {alg.value} INNER JOIN chunks ch ON ch.configuration_id = c.configuration_id AND ch."index" = 1 INNER JOIN results r ON r.chunk_id = ch.chunk_id AND r.type_id = {observable.value} WHERE c.simulation_id = \'{simulation_id}\' AND c.lattice_size = \'{size}\' ORDER BY temperature', con)
        ax.errorbar(data['temperature'], data['tau'], label=f'$N = {size} \\times {size}$')

        ax.set_title(f'{observable.name()} integrated autocorrelation time for a 2D lattice using the {alg.name} algorithm')
        ax.set_xlabel('Temperature $T / \\frac{J}{k_B}$')
        ax.set_ylabel(f'{observable.name()} per spin $\\tau$')
    ax.legend()
    return fig, ax

### Helper for plotting the autocorrelation

In [ ]:
def plot_observable_autocorrelation(con: sqlite3.Connection, simulation_id: int, alg: Algorithm, observable: Observable, size: int) -> tuple[plt.Figure, plt.Axes]:
    fig, ax = plt.subplots()
    temperatures = pd.read_sql_query(f'SELECT c.configuration_id, c.temperature FROM configurations c INNER JOIN metadata m ON c.metadata_id = m.metadata_id AND m.algorithm = {alg.value} WHERE c.simulation_id = {simulation_id} AND c.lattice_size={size}', con)

    for row in [temperatures.iloc[temperatures['temperature'].argmax()], temperatures.iloc[temperatures['temperature'].argmin()]]:
        data = pd.read_sql_query(f'SELECT a.data FROM autocorrelations a WHERE a.configuration_id = {row['configuration_id']} AND a.type_id = {observable.value} LIMIT 1', con).iloc[0]
        vector = Vector.GetRootAsVector(data['data'], 0)
        ax.plot(range(int(vector.DataLength())), vector.DataAsNumpy(), 'x--', label=f'$T = {row['temperature']}$')

        ax.set_title(f'{observable.name()} autocorrelation for a 2D lattice using the {alg.name} algorithm')
        ax.set_xlabel('Monte Carlo time')
        ax.set_ylabel(f'{observable.name()} Autocorrelation Function')

    ax.legend()
    return fig, ax


### Bootstrapping intercept

In [ ]:
def bootstrap_intercept(data: pd.DataFrame, sampling: pd.Series) -> tuple[plt.Figure, plt.Axes]:
    # PLOT DATA POINTS
    fig, ax = plt.subplots(1, 2, gridspec_kw={'width_ratios': [4, 1], 'wspace': 0, 'hspace': 0}, sharey='row', sharex='col')
    ax[0].errorbar(data['size'], data['temperature'], label=f'Temperature $T$ at $\\chi_\\text{{max}}$')

    # BOOTSTRAP INTERCEPT
    boot = pd.DataFrame()
    for i in range(10_000):
        draw = data.sample(len(data), replace=True)
        sizes = draw['size'].to_numpy()

        if not (sizes[0] == sizes).all():
            res = linregress(draw['size'], draw['temperature'])

            new = pd.DataFrame({'slope': [res.slope], 'intercept': [res.intercept]})
            boot = pd.concat([boot, new])

    # GETTING CUTOFFS FOR QUANTILE
    slope_top_percentile = boot['slope'].quantile(0.95)
    slope_bottom_percentile = boot['slope'].quantile(1.0 - 0.95)
    intercept_top_percentile = boot['intercept'].quantile(0.95)
    intercept_bottom_percentile = boot['intercept'].quantile(1.0 - 0.95)

    # FILTER FOR QUANTILE
    slope_quantile = boot[boot['slope'].ge(slope_bottom_percentile) & boot['slope'].le(slope_top_percentile)]
    print(f'Bootstrap slope: {slope_quantile['slope'].mean():.4f} +- {slope_quantile['slope'].std():.4f}')

    intercept_quantile = boot[boot['intercept'].ge(intercept_bottom_percentile) & boot['intercept'].le(intercept_top_percentile)]
    print(f'Bootstrap intercept: {intercept_quantile['intercept'].mean():.4f} +- {intercept_quantile['intercept'].std():.4f}')

    # GET CONFIDENCE INTERVAL FOR INTERCEPT
    cross = intercept_quantile.merge(sampling.rename('x'), how='cross')
    cross['y'] = cross.apply(lambda x: x['x'] * x['slope'] + x['intercept'], axis=1)

    ymax = cross.groupby('x')['y'].max()
    ymin = cross.groupby('x')['y'].min()

    # PLOT REGRESSION
    res = linregress(data['size'], data['temperature'])
    print(f'Linear slope: {res.slope:.4f} +- {intercept_quantile['slope'].std():.4f}')
    print(f'Linear intercept: {res.intercept:.4f} +- {intercept_quantile['intercept'].std():.4f}')

    ax[0].plot(sampling, sampling * res.slope + res.intercept, 'b-', label=f'Fit: $({res.slope:.2f} \\pm {slope_quantile['slope'].std():.2f}) \\cdot x + ({res.intercept:.4f} \\pm {intercept_quantile['intercept'].std():.4f})$')
    ax[0].fill_between(sampling, ymin, ymax, alpha=0.2, label=f'Intercept $0.95$ quantile band')

    # PLOT CENTRAL LIMIT THEOREM
    ax[1].hist(boot['intercept'], bins=50, orientation='horizontal')
    ax[1].axhline(intercept_quantile['intercept'].mean(), color='red', label=f'Mean {intercept_quantile['intercept'].mean():.4f}')
    ax[1].axhspan(intercept_bottom_percentile, intercept_top_percentile, alpha=0.2, label=f'$0.95$ Quantile')

    ax[1].set_title('Distribution of $b$')
    ax[1].set_xlabel('Frequency')
    ax[1].legend()

    return fig, ax

### Helicity modulus intersect

In [ ]:
def line(p1, p2):
    A = (p1[1] - p2[1])
    B = (p2[0] - p1[0])
    C = (p1[0] * p2[1] - p2[0] * p1[1])
    return A, B, -C

def intersection(L1, L2):
    D  = L1[0] * L2[1] - L1[1] * L2[0]
    Dx = L1[2] * L2[1] - L1[1] * L2[2]
    Dy = L1[0] * L2[2] - L1[2] * L2[0]
    if D != 0:
        x = Dx / D
        y = Dy / D
        return x,y
    else:
        return False



def plot_helicity_modulus_intersect(con: sqlite3.Connection, simulation_id: int, alg: Algorithm) -> tuple[plt.Figure, plt.Axes, pd.DataFrame]:
    sizes = []
    temperatures = []
    colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

    fig, ax = plt.subplots()
    for i, size in enumerate(pd.read_sql_query(f'SELECT DISTINCT lattice_size FROM configurations c INNER JOIN metadata m ON c.metadata_id = m.metadata_id AND m.algorithm = {alg.value} WHERE c.simulation_id = {simulation_id}', con)['lattice_size']):
        data = pd.read_sql_query(f'SELECT temperature, e.mean, e.std_dev FROM configurations c INNER JOIN metadata m ON c.metadata_id = m.metadata_id AND m.algorithm = {alg.value} INNER JOIN estimates e ON e.configuration_id = c.configuration_id AND e.type_id = {Observable.HelicityModulus.value} WHERE c.simulation_id = \'{simulation_id}\' AND c.lattice_size = \'{size}\' ORDER BY temperature', con)
        data['reference'] = data['temperature'].transform(lambda x: 2.0 * x / np.pi)
        data['diff'] = data['mean'] - data['reference']

        idx = data['diff'].lt(0).idxmax()
        range = data.loc[idx - 1:idx]

        slope = range['mean'].diff().iloc[1] / range['temperature'].diff().iloc[1]
        intercept = data['mean'].iloc[idx] - slope * data['temperature'].iloc[idx]

        ax.errorbar(data['temperature'], data['mean'], yerr=data['std_dev'], color=colors[i % len(colors)])
        ax.plot(range['temperature'], range['temperature'] * slope + intercept, '--', color=colors[i % len(colors)])

        L1 = line([range['temperature'].iloc[0], range['mean'].iloc[0]], [range['temperature'].iloc[1], range['mean'].iloc[1]])
        L2 = line([0, 0], [1, 2.0 / np.pi])

        R = intersection(L1, L2)
        ax.plot(*R, 'o', label=f'Intersect $N = {size} \\times {size}$', color=colors[i % len(colors)], markersize=4)

        sizes.append(size)
        temperatures.append(R[0])

        ax.set_title(f'{Observable.HelicityModulus.name()} per spin for a 2D lattice using the {alg.name} algorithm')
        ax.set_xlabel('Temperature $T / \\frac{J}{k_B}$')
        ax.set_ylabel(f'{Observable.HelicityModulus.name()} per spin ${Observable.HelicityModulus.symbol()}$')

        ax.legend()
    return fig, ax, pd.DataFrame({'lattice_size': sizes, 'temperature': temperatures})

### Get current run

In [ ]:
con = sqlite3.connect('output/data.db')
simulation_id = pd.read_sql_query('SELECT simulation_id FROM simulations ORDER BY created_at DESC', con).at[0, 'simulation_id']
print(f'The current simulation id is {simulation_id}')

## Simulation

### Energy per Spin [Metropolis]

In [ ]:
fig, _ = plot_observable_per_spin(con, simulation_id, Algorithm.Metropolis, Observable.Energy)
fig.savefig('figures/Metropolis/Energy.pdf', dpi=dpi,  bbox_inches='tight')

In [ ]:
fig, _ = plot_observable_autocorrelation(con, simulation_id, Algorithm.Metropolis, Observable.Energy, 8)
fig.savefig('figures/Metropolis/Energy_Autocorrelation.pdf', dpi=dpi,  bbox_inches='tight')

In [ ]:
fig, _ = plot_observable_integrated_autocorrelation(con, simulation_id, Algorithm.Metropolis, Observable.Energy)
fig.savefig('figures/Metropolis/Energy_IntegratedAutocorrelation.pdf', dpi=dpi,  bbox_inches='tight')

### Energy per Spin [Wolff]

In [ ]:
fig, _ = plot_observable_per_spin(con, simulation_id, Algorithm.Wolff, Observable.Energy)
fig.savefig('figures/Wolff/Energy.pdf', dpi=dpi,  bbox_inches='tight')

In [ ]:
fig, _ = plot_observable_autocorrelation(con, simulation_id, Algorithm.Wolff, Observable.Energy, 8)
fig.savefig('figures/Wolff/Energy_Autocorrelation.pdf', dpi=dpi,  bbox_inches='tight')

In [ ]:
fig, _ = plot_observable_integrated_autocorrelation(con, simulation_id, Algorithm.Wolff, Observable.Energy)
fig.savefig('figures/Wolff/Energy_IntegratedAutocorrelation.pdf', dpi=dpi,  bbox_inches='tight')

### Energy squared per Spin [Metropolis]

In [ ]:
fig, _ = plot_observable_per_spin(con, simulation_id, Algorithm.Metropolis, Observable.EnergySquare)
fig.savefig('figures/Metropolis/EnergySquare.pdf', dpi=dpi,  bbox_inches='tight')

In [ ]:
fig, _ = plot_observable_autocorrelation(con, simulation_id, Algorithm.Metropolis, Observable.EnergySquare, 8)
fig.savefig('figures/Metropolis/EnergySquare_Autocorrelation.pdf', dpi=dpi,  bbox_inches='tight')

In [ ]:
fig, _ = plot_observable_integrated_autocorrelation(con, simulation_id, Algorithm.Metropolis, Observable.EnergySquare)
fig.savefig('figures/Metropolis/EnergySquare_IntegratedAutocorrelation.pdf', dpi=dpi,  bbox_inches='tight')

### Energy squared per Spin [Wolff]

In [ ]:
fig, _ = plot_observable_per_spin(con, simulation_id, Algorithm.Wolff, Observable.EnergySquare)
fig.savefig('figures/Wolff/EnergySquare.pdf', dpi=dpi,  bbox_inches='tight')

In [ ]:
fig, _ = plot_observable_autocorrelation(con, simulation_id, Algorithm.Wolff, Observable.EnergySquare, 8)
fig.savefig('figures/Wolff/EnergySquare_Autocorrelation.pdf', dpi=dpi,  bbox_inches='tight')

In [ ]:
fig, _ = plot_observable_integrated_autocorrelation(con, simulation_id, Algorithm.Wolff, Observable.EnergySquare)
fig.savefig('figures/Wolff/EnergySquare_IntegratedAutocorrelation.pdf', dpi=dpi,  bbox_inches='tight')

### Specific heat per Spin [Metropolis]

In [ ]:
fig, ax = plot_observable_per_spin(con, simulation_id, Algorithm.Metropolis, Observable.SpecificHeat, legend_loc='upper left')
ax.set_yscale('log')
fig.savefig('figures/Metropolis/SpecificHeat.pdf', dpi=dpi,  bbox_inches='tight')

### Specific heat per Spin [Wolff]

In [ ]:
fig, ax = plot_observable_per_spin(con, simulation_id, Algorithm.Wolff, Observable.SpecificHeat, legend_loc='upper left')
ax.set_yscale('log')
fig.savefig('figures/Wolff/SpecificHeat.pdf', dpi=dpi,  bbox_inches='tight')

### Magnetization per Spin [Metropolis]

In [ ]:
fig, _ = plot_observable_per_spin(con, simulation_id, Algorithm.Metropolis, Observable.Magnetization)
fig.savefig('figures/Metropolis/Energy.pdf', dpi=dpi,  bbox_inches='tight')

In [ ]:
fig, _ = plot_observable_autocorrelation(con, simulation_id, Algorithm.Metropolis, Observable.Magnetization, 8)
fig.savefig('figures/Metropolis/Magnetization_Autocorrelation.pdf', dpi=dpi,  bbox_inches='tight')

In [ ]:
fig, _ = plot_observable_integrated_autocorrelation(con, simulation_id, Algorithm.Metropolis, Observable.Magnetization)
fig.savefig('figures/Metropolis/Magnetization_IntegratedAutocorrelation.pdf', dpi=dpi,  bbox_inches='tight')

### Magnetization per Spin [Wolff]

In [ ]:
fig, _ = plot_observable_per_spin(con, simulation_id, Algorithm.Wolff, Observable.Magnetization)
fig.savefig('figures/Wolff/Magnetization.pdf', dpi=dpi,  bbox_inches='tight')

In [ ]:
fig, _ = plot_observable_autocorrelation(con, simulation_id, Algorithm.Wolff, Observable.Magnetization, 8)
fig.savefig('figures/Wolff/Magnetization_Autocorrelation.pdf', dpi=dpi,  bbox_inches='tight')

In [ ]:
fig, _ = plot_observable_integrated_autocorrelation(con, simulation_id, Algorithm.Wolff, Observable.Magnetization)
fig.savefig('figures/Wolff/Magnetization_IntegratedAutocorrelation.pdf', dpi=dpi,  bbox_inches='tight')

### Magnetization squared per Spin [Metropolis]

In [ ]:
fig, _ = plot_observable_per_spin(con, simulation_id, Algorithm.Metropolis, Observable.MagnetizationSquare)
fig.savefig('figures/Metropolis/MagnetizationSquare.pdf', dpi=dpi,  bbox_inches='tight')

In [ ]:
fig, _ = plot_observable_autocorrelation(con, simulation_id, Algorithm.Metropolis, Observable.MagnetizationSquare, 8)
fig.savefig('figures/Metropolis/MagnetizationSquare_Autocorrelation.pdf', dpi=dpi,  bbox_inches='tight')

In [ ]:
fig, _ = plot_observable_integrated_autocorrelation(con, simulation_id, Algorithm.Metropolis, Observable.MagnetizationSquare)
fig.savefig('figures/Metropolis/MagnetizationSquare_IntegratedAutocorrelation.pdf', dpi=dpi,  bbox_inches='tight')

### Magnetization squared per Spin [Wolff]

In [ ]:
fig, _ = plot_observable_per_spin(con, simulation_id, Algorithm.Wolff, Observable.MagnetizationSquare)
fig.savefig('figures/Wolff/MagnetizationSquare.pdf', dpi=dpi,  bbox_inches='tight')

In [ ]:
fig, _ = plot_observable_autocorrelation(con, simulation_id, Algorithm.Wolff, Observable.MagnetizationSquare, 8)
fig.savefig('figures/Wolff/MagnetizationSquare_Autocorrelation.pdf', dpi=dpi,  bbox_inches='tight')

In [ ]:
fig, _ = plot_observable_integrated_autocorrelation(con, simulation_id, Algorithm.Wolff, Observable.MagnetizationSquare)
fig.savefig('figures/Wolff/MagnetizationSquare_IntegratedAutocorrelation.pdf', dpi=dpi,  bbox_inches='tight')

### Magnetic Susceptibility per Spin [Metropolis]

In [ ]:
fig, ax = plot_observable_per_spin(con, simulation_id, Algorithm.Metropolis, Observable.MagneticSusceptibility)
ax.set_yscale('log')
fig.savefig('figures/Metropolis/MagneticSusceptibility.pdf', dpi=dpi,  bbox_inches='tight')

### Magnetic Susceptibility per Spin [Wolff]

In [ ]:
fig, ax = plot_observable_per_spin(con, simulation_id, Algorithm.Wolff, Observable.MagneticSusceptibility)
ax.set_yscale('log')
fig.savefig('figures/Wolff/MagneticSusceptibility.pdf', dpi=dpi,  bbox_inches='tight')

### Helicity modulus per Spin [Metropolis]

In [ ]:
sampling = np.linspace(0, 1.5)
fig, ax = plot_observable_per_spin(con, simulation_id, Algorithm.Metropolis, Observable.HelicityModulus)
ax.plot(sampling, 2.0 * sampling / np.pi, '-', label='$\\frac{2}{\\pi} T$')
ax.legend()
fig.savefig('figures/Metropolis/HelicityModulus.pdf', dpi=dpi,  bbox_inches='tight')

### Helicity modulus per Spin [Wolff]

In [ ]:
sampling = np.linspace(0, 1.5)
fig, ax = plot_observable_per_spin(con, simulation_id, Algorithm.Wolff, Observable.HelicityModulus)
ax.plot(sampling, 2.0 * sampling / np.pi, '-', label='$\\frac{2}{\\pi} T$')
ax.legend()
fig.savefig('figures/Wolff/HelicityModulus.pdf', dpi=dpi,  bbox_inches='tight')

### Cluster size [Wolff]

In [ ]:
fig, ax = plot_observable_per_spin(con, simulation_id, Algorithm.Wolff, Observable.ClusterSize)
fig.savefig('figures/Wolff/ClusterSize.pdf', dpi=dpi,  bbox_inches='tight')

### Peak integrated autocorrelation time comparison

In [ ]:
fig, ax = plt.subplots()
observable = Observable.Energy

for _, row in pd.read_sql_query(f'SELECT metadata_id, algorithm FROM metadata m WHERE simulation_id = {simulation_id}', con).iterrows():
    data = pd.read_sql_query(f'SELECT c.lattice_size, MAX(r.tau) FROM configurations c INNER JOIN chunks chk ON c.configuration_id = chk.configuration_id INNER JOIN results r ON chk.chunk_id = r.chunk_id AND r.type_id = {observable.value} AND chk."index" = 1 WHERE c.simulation_id = {simulation_id} AND c.metadata_id = {row['metadata_id']} AND c.temperature BETWEEN 0.8 AND 1.2 GROUP BY c.lattice_size', con);
    ax.plot(data['lattice_size'], data['MAX(r.tau)'], 'x--', label=f'{Algorithm(row['algorithm']).name}')

ax.legend()


## Analysis

### TC from Magnetic susceptibility [Metropolis]

In [ ]:
sampling = pd.Series(np.linspace(0, 0.25, num=100))

data = get_observable_max(con, simulation_id, Algorithm.Metropolis, Observable.MagneticSusceptibility)
data['size'] = data['lattice_size'].transform(lambda x: 1.0 / np.log(x)**2)

fig, ax = bootstrap_intercept(data, sampling)
ax[0].set_title('Critical temperature $T_C$ for the XY model on a 2D lattice')
ax[0].set_xlabel('Scaling $(\\log{L})^{-2}$')
ax[0].set_ylabel('Temperature $T / \\frac{J}{k_B}$')
ax[0].legend()

fig.savefig('figures/Metropolis/Critical_Temperature.pdf', dpi=dpi,  bbox_inches='tight')

### TC from Magnetic susceptibility [Wolff]

In [ ]:
sampling = pd.Series(np.linspace(0, 0.25, num=100))

data = get_observable_max(con, simulation_id, Algorithm.Wolff, Observable.MagneticSusceptibility)
data['size'] = data['lattice_size'].transform(lambda x: 1.0 / np.log(x)**2)

fig, ax = bootstrap_intercept(data, sampling)
ax[0].set_title('Critical temperature $T_C$ for the XY model on a 2D lattice')
ax[0].set_xlabel('Scaling $(\\log{L})^{-2}$')
ax[0].set_ylabel('Temperature $T / \\frac{J}{k_B}$')
ax[0].legend()

fig.savefig('figures/Wolff/Critical_Temperature.pdf', dpi=dpi,  bbox_inches='tight')

### TC from Helicity modulus [Metropolis]

In [ ]:
sampling = np.linspace(0, 1.5)

fig, ax, data = plot_helicity_modulus_intersect(con, simulation_id, Algorithm.Metropolis)
ax.plot(sampling, 2.0 * sampling / np.pi, 'k-', label='$\\frac{2}{\\pi T}$', zorder=0)
ax.set_xlim([0.85, 0.95])
ax.set_ylim([0.5, 0.65])
fig.savefig('figures/Metropolis/HelicityModulus_Intersect.pdf', dpi=dpi,  bbox_inches='tight')

In [ ]:
sampling = pd.Series(np.linspace(0, 0.4, num=100))
data['size'] = data['lattice_size'].transform(lambda x: 1.0 / x)

fig, ax = bootstrap_intercept(data, sampling)
ax[0].set_title('Critical temperature $T_C$ for the XY model on a 2D lattice')
ax[0].set_xlabel('Scaling $(\\log{L})^{-2}$')
ax[0].set_ylabel('Temperature $T / \\frac{J}{k_B}$')
ax[0].legend()

### TC from Helicity modulus [Wolff]

In [ ]:
sampling = np.linspace(0, 1.5)

fig, ax, data = plot_helicity_modulus_intersect(con, simulation_id, Algorithm.Wolff)
ax.plot(sampling, 2.0 * sampling / np.pi, 'k-', label='$\\frac{2}{\\pi T}$', zorder=0)
ax.set_xlim([0.85, 0.95])
ax.set_ylim([0.5, 0.65])
fig.savefig('figures/Wolff/HelicityModulus_Intersect.pdf', dpi=dpi,  bbox_inches='tight')

In [ ]:
sampling = pd.Series(np.linspace(0, 0.4, num=100))
data['size'] = data['lattice_size'].transform(lambda x: 1.0 / x)

fig, ax = bootstrap_intercept(data, sampling)
ax[0].set_title('Critical temperature $T_C$ for the XY model on a 2D lattice')
ax[0].set_xlabel('Scaling $(\\log{L})^{-2}$')
ax[0].set_ylabel('Temperature $T / \\frac{J}{k_B}$')
ax[0].legend()

## Vortices

In [ ]:
vortices = pd.read_sql_query(f'SELECT vortex_id, lattice_size FROM vortices WHERE simulation_id = {simulation_id} ORDER BY lattice_size DESC', con)
data = pd.read_sql_query(f'SELECT temperature, spins FROM vortex_results WHERE vortex_id = {vortices.iloc[0]['vortex_id']} ORDER BY vortex_id ASC, sweeps ASC', con)

fig, ax = plt.subplots(figsize=(16, 9), dpi=1280/16)
ax.set_box_aspect(1)

def animate(n: int):
    ax.cla()
    row = data.iloc[n]
    
    vector = Vector.GetRootAsVector(row['spins'], 0)
    spins = vector.DataAsNumpy();

    u = np.reshape(np.cos(spins), (-1, vortices.iloc[0]['lattice_size']))
    v = np.reshape(np.sin(spins), (-1, vortices.iloc[0]['lattice_size']))

    x = np.arange(1, vortices.iloc[0]['lattice_size'] + 1, 1)
    y = np.arange(1, vortices.iloc[0]['lattice_size'] + 1, 1)

    X, Y = np.meshgrid(x, y)
    ax.quiver(X, Y, u, v, spins, scale=40, pivot='middle', cmap='hsv')
    ax.set_title(f'Temperature $T={row['temperature']:.2f}$')

    print(f'\rFrame: {n}/{len(data)}', end='', flush=True)
    return fig

anim = FuncAnimation(fig = fig, func = animate, frames = len(data), interval = 1, repeat = False)

FFwriter = FFMpegWriter(fps=60, extra_args=['-vcodec', 'libx264', '-preset', 'veryslow', '-pix_fmt', 'yuv420p', '-crf', '23'])
anim.save('figures/vortices.mp4', writer=FFwriter)